# Make table of warming years from zarr data based on 21 year moving window approach

In [1]:
import pandas as pd
import numpy as np
import os.path
from warming_years import calc_warming_years_rolling_mean
from file_control import gmst_table_dir
from file_control import warming_year_table_dir

In [2]:
# Set output filename
outfile = os.path.join(warming_year_table_dir, 'warming_years_zarr_21yr_window.csv')

# Import GMST data
tab = pd.read_csv(os.path.join(gmst_table_dir, 'CMIP6_GMST_table_all.csv'))

# get all combinations of model, member, scenario
# remove historical since those years will be included in the other scenarios
mms = tab.loc[tab['scenario'] != 'historical', ['model','member','scenario'
                                               ]].drop_duplicates()

# set window size and warming levels
window_size = 21
wl = np.arange(1,4.51,.5)

In [3]:
# calculate warming years for each warming level

for i in wl:
    print(i)
    
    tab1 = calc_warming_years_rolling_mean(mms, i, window_size, 2100)
    tab1 = tab1.rename(columns={'warming_year':('Yr' + str(i))})
    
    if i == wl[0]:
        outdf = tab1
    else:
        outdf = pd.merge(outdf, tab1, how='outer', on = ['model','member',
                                                         'scenario'])

outdf 


1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5


,model,member,scenario,Yr1.0,Yr1.5,Yr2.0,Yr2.5,Yr3.0,Yr3.5,Yr4.0,Yr4.5
0,GFDL-CM4,r1i1p1f1,ssp245,2015.0,2031.0,2050.0,2071.0,NaN,NaN,NaN,NaN
1,GFDL-CM4,r1i1p1f1,ssp585,2015.0,2030.0,2042.0,2051.0,2060.0,2070.0,2079.0,2087.0
2,GFDL-ESM4,r1i1p1f1,ssp119,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GFDL-ESM4,r1i1p1f1,ssp126,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GFDL-ESM4,r1i1p1f1,ssp245,2019.0,2046.0,2073.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1293,ACCESS-CM2,r4i1p1f1,ssp585,2012.0,2026.0,2037.0,2047.0,2055.0,2062.0,2070.0,2076.0
1294,ACCESS-CM2,r5i1p1f1,ssp126,2012.0,2026.0,2046.0,NaN,NaN,NaN,NaN,NaN
1295,ACCESS-CM2,r5i1p1f1,ssp245,2013.0,2027.0,2040.0,2052.0,2066.0,2088.0,NaN,NaN
1296,ACCESS-CM2,r5i1p1f1,ssp370,2014.0,2027.0,2037.0,2048.0,2059.0,2068.0,2077.0,2085.0


In [4]:
# save the warming years table
outdf.to_csv(outfile, index = False)